# Complete t-SNE Analysis: CLIP Features & Adversarial Attacks

This notebook contains:

**Part 1: CLIP Loss Function Comparison**
- Vanilla CLIP vs ArcFace vs SigLip vs ArcFaceSigmoid
- Both Euclidean AND Cosine metrics

**Part 2: UNet Adversarial Feature Analysis**
- Clean vs UNet Vanilla vs UNet Contrastive
- Feature disruption metrics

In [ ]:
# Cell 1: GPU + Repo Setup
!nvidia-smi
%cd /content

import os
if not os.path.exists("MFCLIP_acv"):
    !git clone -b hamza/discrim https://github.com/1hamzaiqbal/MFCLIP_acv

%cd MFCLIP_acv
!git fetch --all
!git reset --hard origin/hamza/discrim

In [ ]:
# Cell 2: Install Dependencies
!pip install -q torch torchvision timm einops yacs tqdm opencv-python scikit-learn scipy pyyaml ruamel.yaml pytorch-ignite foolbox pandas matplotlib seaborn wilds ftfy

In [ ]:
# Cell 3: Mount Drive + Setup Dataset
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

from torchvision.datasets import OxfordIIITPet
from torchvision import transforms
from pathlib import Path
import os

# Dataset setup
root = Path("/content/data/oxford_pets")
root.mkdir(parents=True, exist_ok=True)
_ = OxfordIIITPet(root=str(root), download=True, transform=transforms.ToTensor())

%cd /content
if not os.path.exists("/content/data/oxford_pets/images"):
    print("Downloading images and annotations...")
    !wget -q https://www.robots.ox.ac.uk/~vgg/data/pets/data/images.tar.gz
    !wget -q https://www.robots.ox.ac.uk/~vgg/data/pets/data/annotations.tar.gz
    !tar -xf images.tar.gz -C /content/data/oxford_pets
    !tar -xf annotations.tar.gz -C /content/data/oxford_pets
    !rm -f images.tar.gz annotations.tar.gz

DRIVE_ROOT = "/content/drive/MyDrive/grad/comp_vision/hanson_loss/oxford_pets"
print(f"\nDrive root: {DRIVE_ROOT}")
print("\nAvailable checkpoints:")
if os.path.exists(DRIVE_ROOT):
    for f in sorted(os.listdir(DRIVE_ROOT)):
        if f.endswith('.pth') or f.endswith('.pt'):
            print(f"  {f}")

In [ ]:
# Cell 4: Setup Python Path + Extract CLIP Features
import sys
sys.path.insert(0, "/content/MFCLIP_acv/lpclip")
sys.path.insert(0, "/content/MFCLIP_acv")
os.makedirs("/content/MFCLIP_acv/lpclip/datasets", exist_ok=True)
open("/content/MFCLIP_acv/lpclip/datasets/__init__.py", "a").close()

%cd /content/MFCLIP_acv
FEAT_DIR = "/content/MFCLIP_acv/clip_feat"

print("Extracting vanilla CLIP features...")
!python -m lpclip.feat_extractor --split test --root /content/data --seed 1 \
    --dataset-config-file configs/datasets/oxford_pets.yaml \
    --config-file configs/trainers/CoOp/rn50_val.yaml \
    --output-dir {FEAT_DIR} --eval-only

print("\nFeature extraction complete!")

---
# PART 1: CLIP Loss Function Comparison
---

In [ ]:
# Cell 5: Load Features + Define Helper Functions
import numpy as np
import matplotlib.pyplot as plt
import torch
import pandas as pd
from sklearn.manifold import TSNE
from sklearn.metrics import silhouette_score
from scipy.spatial.distance import pdist, cosine
from tqdm import tqdm

FEAT_DIR = "/content/MFCLIP_acv/clip_feat/OxfordPets"
OUT_DIR = "/content/tsne_results"
os.makedirs(OUT_DIR, exist_ok=True)

# Load vanilla CLIP features
d = np.load(f"{FEAT_DIR}/test.npz")
X_vanilla = d["feature_list"].astype(np.float32)
y = d["label_list"].astype(np.int64)
print(f"Loaded {len(X_vanilla)} samples, {len(np.unique(y))} classes, dim={X_vanilla.shape[1]}")

def torch_load_any(path):
    sd = torch.load(path, map_location="cpu")
    return sd["state_dict"] if isinstance(sd, dict) and "state_dict" in sd else sd

def guess_head_weight(sd, in_dim):
    cands = [(k,v) for k,v in sd.items() if isinstance(v, torch.Tensor) and v.ndim==2]
    for k,v in cands:
        if v.shape[0] == in_dim:
            return v.numpy(), k
    return None, None

def cosine_project(X, W):
    Xn = X / np.maximum(np.linalg.norm(X, axis=1, keepdims=True), 1e-12)
    Wn = W / np.maximum(np.linalg.norm(W, axis=0, keepdims=True), 1e-12)
    return Xn @ Wn

In [ ]:
# Cell 6: Define Checkpoint Paths (EDIT AS NEEDED)
CHECKPOINTS = {
    "ArcFace": f"{DRIVE_ROOT}/RN50_ArcFace_oxford_pets.pth",
    "ArcFaceSigmoid": f"{DRIVE_ROOT}/RN50_ArcFaceSigmoid_300ep.pth",
    "SigLip": f"{DRIVE_ROOT}/RN50_SigLipHead_300ep.pth",
}

# Build feature dict
all_features = {"Vanilla CLIP": X_vanilla}

for name, path in CHECKPOINTS.items():
    if os.path.exists(path):
        sd = torch_load_any(path)
        W, key = guess_head_weight(sd, X_vanilla.shape[1])
        if W is not None:
            all_features[name] = cosine_project(X_vanilla, W)
            print(f"OK {name} - using {key}")
        else:
            print(f"SKIP {name} - no weights found")
    else:
        print(f"MISSING {name}")

print(f"\nTotal variants: {len(all_features)}")

In [ ]:
# Cell 7: Run t-SNE with BOTH Euclidean and Cosine metrics
results_euclidean = {}
results_cosine = {}

for name, X in all_features.items():
    print(f"\nProcessing {name}...")
    
    # Euclidean t-SNE
    print("  Running Euclidean t-SNE...")
    Z_euc = TSNE(n_components=2, perplexity=30, random_state=42, init='pca', n_iter=1000).fit_transform(X)
    sil_euc = silhouette_score(Z_euc, y)
    results_euclidean[name] = {"Z": Z_euc, "sil": sil_euc}
    
    # Cosine t-SNE
    print("  Running Cosine t-SNE...")
    X_norm = X / np.linalg.norm(X, axis=1, keepdims=True)
    Z_cos = TSNE(n_components=2, perplexity=30, metric='cosine', random_state=42, init='random', n_iter=1000).fit_transform(X_norm)
    sil_cos = silhouette_score(X_norm, y, metric='cosine')
    results_cosine[name] = {"Z": Z_cos, "sil": sil_cos}
    
    print(f"  Euclidean Sil: {sil_euc:.4f}, Cosine Sil: {sil_cos:.4f}")

print("\nDone!")

In [ ]:
# Cell 8: Visualize BOTH metrics side by side
n = len(all_features)
fig, axes = plt.subplots(2, n, figsize=(5*n, 10), dpi=120)
cmap = plt.cm.get_cmap('tab20', len(np.unique(y)))

for idx, name in enumerate(all_features.keys()):
    # Euclidean row
    ax = axes[0, idx]
    Z = results_euclidean[name]["Z"]
    sil = results_euclidean[name]["sil"]
    ax.scatter(Z[:, 0], Z[:, 1], c=y, s=8, alpha=0.7, cmap=cmap)
    ax.set_title(f"{name}\nEuclidean Sil: {sil:.3f}", fontsize=10)
    ax.set_xticks([]); ax.set_yticks([])
    if idx == 0:
        ax.set_ylabel("EUCLIDEAN", fontsize=12, fontweight='bold')
    
    # Cosine row
    ax = axes[1, idx]
    Z = results_cosine[name]["Z"]
    sil = results_cosine[name]["sil"]
    ax.scatter(Z[:, 0], Z[:, 1], c=y, s=8, alpha=0.7, cmap=cmap)
    ax.set_title(f"{name}\nCosine Sil: {sil:.3f}", fontsize=10)
    ax.set_xticks([]); ax.set_yticks([])
    if idx == 0:
        ax.set_ylabel("COSINE", fontsize=12, fontweight='bold')

plt.suptitle("CLIP Features: Euclidean vs Cosine Metrics", fontsize=14, y=1.01)
plt.tight_layout()
plt.savefig(f"{OUT_DIR}/clip_euclidean_vs_cosine.png", dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
# Cell 9: Summary Table for CLIP Loss Comparison
print("=" * 70)
print("CLIP LOSS FUNCTION COMPARISON")
print("=" * 70)
print(f"{'Loss':<20} {'Euclidean Sil':>15} {'Cosine Sil':>15} {'Winner':>12}")
print("-" * 70)
for name in all_features.keys():
    euc = results_euclidean[name]["sil"]
    cos = results_cosine[name]["sil"]
    winner = "Cosine" if cos > euc else "Euclidean"
    print(f"{name:<20} {euc:>15.4f} {cos:>15.4f} {winner:>12}")
print("=" * 70)
print("\nNote: ArcFace/SigLip optimize for angular separation,")
print("so Cosine silhouette is the 'correct' metric for them.")

---
# PART 2: UNet Adversarial Feature Analysis
---

In [ ]:
# Cell 10: Load Surrogate Model for Feature Extraction
%cd /content/MFCLIP_acv

import torch
import torch.nn as nn
from ruamel.yaml import YAML

from model import UNetLikeGenerator as UNet
from utils.util import setup_cfg, Model
from dass.engine import build_trainer
from loss.head.head_def import HeadFactory

import trainers.zsclip, trainers.coop, trainers.cocoop
import datasets.oxford_pets, datasets.oxford_flowers, datasets.food101

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

class Args:
    root = "/content/data"
    dataset = "oxford_pets"
    config_file = "configs/trainers/CoOp/rn50.yaml"
    dataset_config_file = "configs/datasets/oxford_pets.yaml"
    trainer = "ZeroshotCLIP"
    head = "ArcFace"
    output_dir = "output"
    opts = []
    gpu = 0
    device = "cuda:0"
    resume = ""
    seed = -1
    source_domains = None
    target_domains = None
    transforms = None
    backbone = ""
    bs = 64
    ratio = 1.0

args = Args()
cfg = setup_cfg(args)
trainer = build_trainer(cfg)

yaml_parser = YAML(typ='safe')
config = yaml_parser.load(open('configs/data.yaml', 'r'))
config['num_classes'] = trainer.dm.num_classes
config['output_dim'] = 1024
head_factory = HeadFactory(args.head, config)

clip_backbone = trainer.clip_model.visual
normalize = transforms.Normalize([0.48145466, 0.4578275, 0.40821073], [0.26862954, 0.26130258, 0.27577711])
backbone = nn.Sequential(normalize, clip_backbone)

surrogate = Model(backbone, head_factory).to(device)

# Load surrogate weights
SURROGATE_PATH = f"{DRIVE_ROOT}/RN50_ArcFace_oxford_pets.pth"
if os.path.exists(SURROGATE_PATH):
    surrogate.load_state_dict(torch.load(SURROGATE_PATH, map_location=device))
    print(f"Surrogate loaded from {SURROGATE_PATH}")
surrogate.eval()

test_loader = trainer.test_loader
NUM_CLASSES = trainer.dm.num_classes
print(f"Test set: {len(test_loader.dataset)} samples, {NUM_CLASSES} classes")

In [ ]:
# Cell 11: Load UNet Generators
UNET_VANILLA_PATH = f"{DRIVE_ROOT}/unet-vanilla-pets.pt"
UNET_CONTRASTIVE_PATH = f"{DRIVE_ROOT}/unet-contrastive-pets.pt"

unet_vanilla = None
unet_contrastive = None

if os.path.exists(UNET_VANILLA_PATH):
    unet_vanilla = UNet().to(device)
    unet_vanilla.load_state_dict(torch.load(UNET_VANILLA_PATH, map_location=device))
    unet_vanilla.eval()
    print(f"OK UNet Vanilla")
else:
    print(f"MISSING UNet Vanilla at {UNET_VANILLA_PATH}")

if os.path.exists(UNET_CONTRASTIVE_PATH):
    unet_contrastive = UNet().to(device)
    unet_contrastive.load_state_dict(torch.load(UNET_CONTRASTIVE_PATH, map_location=device))
    unet_contrastive.eval()
    print(f"OK UNet Contrastive")
else:
    print(f"MISSING UNet Contrastive at {UNET_CONTRASTIVE_PATH}")

In [ ]:
# Cell 12: Extract Clean and Adversarial Features
NUM_SAMPLES = 1000
EPS = 16/255.

def extract_features(loader, backbone, generator=None, num_samples=1000, eps=16/255.):
    backbone.eval()
    if generator is not None:
        generator.eval()
    
    all_features = []
    all_labels = []
    count = 0
    
    with torch.no_grad():
        for batch in tqdm(loader, desc="Extracting"):
            images = batch['img'].to(device)
            labels = batch['label'].to(device)
            
            if generator is not None:
                try:
                    target_labels = torch.randint(0, NUM_CLASSES, labels.shape).to(device)
                    noise = generator(images, target_labels)
                except TypeError:
                    noise = generator(images)
                noise = torch.clamp(noise, -eps, eps)
                images = torch.clamp(images + noise, 0, 1)
            
            features = backbone(images)
            all_features.append(features.cpu().numpy())
            all_labels.append(labels.cpu().numpy())
            
            count += len(labels)
            if count >= num_samples:
                break
    
    return np.concatenate(all_features)[:num_samples], np.concatenate(all_labels)[:num_samples]

# Extract features
print("Extracting clean features...")
feat_clean, labels_adv = extract_features(test_loader, surrogate.backbone, None, NUM_SAMPLES)

adv_features = {"Clean": feat_clean}

if unet_vanilla is not None:
    print("Extracting UNet Vanilla adversarial features...")
    feat_unet_vanilla, _ = extract_features(test_loader, surrogate.backbone, unet_vanilla, NUM_SAMPLES, EPS)
    adv_features["UNet Vanilla"] = feat_unet_vanilla
    del unet_vanilla
    torch.cuda.empty_cache()

if unet_contrastive is not None:
    print("Extracting UNet Contrastive adversarial features...")
    feat_unet_contrastive, _ = extract_features(test_loader, surrogate.backbone, unet_contrastive, NUM_SAMPLES, EPS)
    adv_features["UNet Contrastive"] = feat_unet_contrastive
    del unet_contrastive
    torch.cuda.empty_cache()

print(f"\nExtracted: {list(adv_features.keys())}")

In [ ]:
# Cell 13: t-SNE for Adversarial Features
# Combine for joint t-SNE
all_feats = []
all_names = []
indices = []
start = 0

for name, feats in adv_features.items():
    all_feats.append(feats)
    indices.append((name, start, start + len(feats)))
    start += len(feats)

combined = np.concatenate(all_feats)
print(f"Running joint t-SNE on {combined.shape[0]} samples...")
Z_all = TSNE(n_components=2, perplexity=30, random_state=42, init='pca', n_iter=1000).fit_transform(combined)

# Visualize
n = len(adv_features)
fig, axes = plt.subplots(1, n+1, figsize=(5*(n+1), 5), dpi=120)
cmap = plt.cm.get_cmap('tab20', len(np.unique(labels_adv)))

for idx, (name, start, end) in enumerate(indices):
    ax = axes[idx]
    Z = Z_all[start:end]
    ax.scatter(Z[:, 0], Z[:, 1], c=labels_adv, s=8, alpha=0.7, cmap=cmap)
    ax.set_title(f"{name}\n(colored by class)", fontsize=10)
    ax.set_xticks([]); ax.set_yticks([])

# Overlay plot
ax = axes[-1]
colors = plt.cm.Set1(np.linspace(0, 1, len(indices)))
markers = ['o', 's', '^', 'D', 'v']
for idx, ((name, start, end), color, marker) in enumerate(zip(indices, colors, markers)):
    Z = Z_all[start:end]
    ax.scatter(Z[:, 0], Z[:, 1], c=[color], s=10, alpha=0.5, marker=marker, label=name)
ax.legend(loc='upper right', fontsize=8)
ax.set_title("All Overlaid\n(colored by variant)", fontsize=10)
ax.set_xticks([]); ax.set_yticks([])

plt.suptitle("t-SNE: Clean vs UNet Adversarial Features", fontsize=14, y=1.02)
plt.tight_layout()
plt.savefig(f"{OUT_DIR}/unet_adversarial_tsne.png", dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
# Cell 14: Feature Disruption Metrics
print("=" * 70)
print("FEATURE DISRUPTION METRICS")
print("=" * 70)
print(f"{'Generator':<20} {'Cosine Dist':>15} {'L2 Dist':>15}")
print("-" * 70)

disruption_results = []
for name, feats in adv_features.items():
    if name == "Clean":
        continue
    
    # Cosine distance per sample
    cos_dists = [cosine(feat_clean[i], feats[i]) for i in range(len(feats))]
    cos_mean, cos_std = np.mean(cos_dists), np.std(cos_dists)
    
    # L2 distance
    l2_dists = np.linalg.norm(feat_clean - feats, axis=1)
    l2_mean, l2_std = np.mean(l2_dists), np.std(l2_dists)
    
    print(f"{name:<20} {cos_mean:>12.4f}+-{cos_std:.3f} {l2_mean:>12.2f}+-{l2_std:.2f}")
    disruption_results.append({"Generator": name, "Cosine": cos_mean, "L2": l2_mean})

print("=" * 70)
print("Higher = more disruption (better attack)")

In [ ]:
# Cell 15: Save All Results to Drive
import shutil

SAVE_DIR = f"{DRIVE_ROOT}/tsne_complete_results"
os.makedirs(SAVE_DIR, exist_ok=True)

files = [
    f"{OUT_DIR}/clip_euclidean_vs_cosine.png",
    f"{OUT_DIR}/unet_adversarial_tsne.png",
]

print(f"Saving to {SAVE_DIR}...")
for f in files:
    if os.path.exists(f):
        shutil.copy(f, SAVE_DIR)
        print(f"  OK {os.path.basename(f)}")

print("\nDone! All results saved.")